In [1]:
!pip install keybert
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.

In [2]:
import pandas as pd
from keybert import KeyBERT

2025-06-03 18:11:52.211610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748974312.431394      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748974312.501418      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
file_path = '/kaggle/input/us-news/us_newsarticle_August_1'

with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [4]:
chunks = [c.strip() for c in raw_text.split('-----') if c.strip()]

data = []
for i in range(0, len(chunks), 3):
    try:
        heading = chunks[i]
        date = chunks[i + 1]
        content = chunks[i + 2]
        data.append((heading, date, content))
    except IndexError:
        pass  

df = pd.DataFrame(data, columns=['heading', 'date', 'content'])

In [5]:
# Step 5: Extract keywords using KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

def extract_keywords(text, top_n=5):
    keywords = kw_model.extract_keywords(text, 
                                         keyphrase_ngram_range=(1, 2), 
                                         stop_words='english', 
                                         top_n=top_n)
    return [kw[0] for kw in keywords]

df['keywords'] = df['content'].apply(lambda x: extract_keywords(x))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
df.to_csv('us_news_cleaned_with_keywords.csv', index=False)

print(df.head())

                                             heading                date  \
0  Melania Trump's girl-on-girl photos from racy ...  August 1   12:57am   
1  Ex-Venezuelan narcotics officials face drug-tr...  August 1   11:30pm   
2  Props used in iconic movies to hit the auction...   August 1   6:17pm   
3  Zika outbreak prompts travel warning for parts...   August 1   4:04pm   
4  Iranians outraged after porn star visits for n...   August 1   3:18pm   

                                             content  \
0  Here’s the nation’s would-be first lady — and ...   
1  Two former top Venezuelan anti-narcotics offic...   
2  View SlideshowThis auction is the reel deal!A ...   
3  A growing Zika outbreak in Florida has forced ...   
4  Iranians are hot and bothered over a busty por...   

                                            keywords  
0  [pictures melania, modeling melania, melania f...  
1  [indicted brooklyn, narcotics officials, indic...  
2  [movie memorabilia, auction movie, mem